In [1]:
import nltk
import time

path = ""/Users/boli/Desktop/"nlp/HW2/clothing_shoes_jewelry.txt"

rfile = open(path,"r") 
contents = rfile.read()
rfile.close()
print(len(contents))

139641164


In [2]:
lines = contents.splitlines()
print(len(lines))
print(lines[:10])

2786318
['reviewerID:A1KLRMWW2FWPL4', 'asin:0000031887', 'reviewerName:Amazon Customer "cameramom"', 'helpful:[0, 0]', "reviewText:This is a great tutu and at a really great price. It doesn't look cheap at all. I'm so glad I looked on Amazon and found such an affordable tutu that isn't made poorly. A++", 'overall:5.0', 'summary:Great tutu-  not cheaply made', 'unixReviewTime:1297468800', 'reviewTime:02 12, 2011', '']


In [3]:
# Remove title "reviewText:"
reviewText = [r[11:] for r in lines if "reviewText" in r ] # r[11:] is contents after 'reviewText:'
print(len(reviewText))
print(reviewText[:3])

278677
["This is a great tutu and at a really great price. It doesn't look cheap at all. I'm so glad I looked on Amazon and found such an affordable tutu that isn't made poorly. A++", 'I bought this for my 4 yr old daughter for dance class, she wore it today for the first time and the teacher thought it was adorable. I bought this to go with a light blue long sleeve leotard and was happy the colors matched up great. Price was very good too since some of these go for over $15.00 dollars.', 'What can I say... my daughters have it in orange, black, white and pink and I am thinking to buy for they the fuccia one. It is a very good way for exalt a dancer outfit: great colors, comfortable, looks great, easy to wear, durables and little girls love it. I think it is a great buy for costumer and play too.']


In [30]:
str_join = ",".join(reviewText)

print(len(str_join))
print(str_join[:300])

88129771
This is a great tutu and at a really great price. It doesn't look cheap at all. I'm so glad I looked on Amazon and found such an affordable tutu that isn't made poorly. A++,I bought this for my 4 yr old daughter for dance class, she wore it today for the first time and the teacher thought it was ado


In [5]:
# sentences tokenize
textsplit = nltk.sent_tokenize(str_join)

print(len(textsplit))
print(textsplit[:20])

907576
['This is a great tutu and at a really great price.', "It doesn't look cheap at all.", "I'm so glad I looked on Amazon and found such an affordable tutu that isn't made poorly.", 'A++,I bought this for my 4 yr old daughter for dance class, she wore it today for the first time and the teacher thought it was adorable.', 'I bought this to go with a light blue long sleeve leotard and was happy the colors matched up great.', 'Price was very good too since some of these go for over $15.00 dollars.,What can I say... my daughters have it in orange, black, white and pink and I am thinking to buy for they the fuccia one.', 'It is a very good way for exalt a dancer outfit: great colors, comfortable, looks great, easy to wear, durables and little girls love it.', 'I think it is a great buy for costumer and play too.,We bought several tutus at once, and they are got high reviews.', 'Sturdy and seemingly well-made.', 'The girls have been wearing them regularly, including out to play, and the 

In [6]:
########### Analyze Sentence Start: ###########

In [7]:
# Extract Imperative & Question

from pycorenlp import StanfordCoreNLP
from pyparsing import OneOrMore, nestedExpr

nlp = StanfordCoreNLP('http://localhost:9000')
question_sentence = []
imperative_sentence = []

def analyze_sentence(text):  # using StanfordCoreNLP
    output = nlp.annotate(text, properties={
    'annotators': 'parse',
    'outputFormat': 'json'
    })
    
    if "ADJP" in output['sentences'][0]['parse']: # Adj Phrase Sentences
        data = OneOrMore(nestedExpr()).parseString(output['sentences'][0]['parse'])  
        
        if ('?' in text or "SBARQ" in data[0][1][0] or "SQ" in data[0][1][0]) and ('!' not in text): # Question
            question_sentence.append(text)
        elif ('!' in text and '?' not in text) and ("S" == data[0][1][0] and "VP" == data[0][1][1][0]): # Imperative
            imperative_sentence.append(text)
        else:
            return None
            
start = time.time()

index = 0
while index < len(textsplit):
    try:
        analyze_sentence(textsplit[index])
    except:
        continue
    finally:
        index+=1
        
        
print('time:\t ' , time.time()-start)
print("-----Question:")
print(len(question_sentence))
print(question_sentence[:30])
print()
print("-----Imperative:")
print(len(imperative_sentence))
print(imperative_sentence[:30])

time:	  142444.28042697906
-----Question:
3682
["I would definitely try other languages after seeing just how easy it is to use this stuff.What didn't I like?For starters the headphones and microphone that came with it are NOT quality, they are not comfortable and my ears got really hot.", 'I was slightly disappointed that the headset headphones did not play the audio, and instead the computer speakers played the audio, so why do I need a headset for just the microphone?', 'Are you just the slightest bit interested in what your current altitude may be?', 'But I had to laugh when I realized I could not enter my own combination numbers.....the maker actually gives you the combination you have to use on your lock and then you get to memorize THEIR numbers.....how stupid are we getting?', "The watch face was a little smaller than I expected, but nothing wrong with that.,Love it.it's a bit edgy and bothering at first but soon its very comfortable.does it job.Considering buying?", 'She says 

In [8]:
# Avg length of imperative sentence
imp_len = 0
for i in imperative_sentence:
    imp_len += len(i)

imp_avg_len = imp_len / len(imperative_sentence)

print(imp_avg_len)

90.41437444543034


In [9]:
# Avg length of question sentence
ques_len = 0
for i in question_sentence:
    ques_len += len(i)

ques_avg_len = ques_len / len(question_sentence)

print(ques_avg_len)

136.81966322650734


In [49]:
ques_token_len = 0
ques_tokens = [nltk.word_tokenize(t) for t in question_sentence]

for i in ques_tokens:
    ques_token_len += len(i)

ques_avg_token = ques_token_len / len(ques_tokens)

print(ques_avg_token)

30.422053231939163


In [48]:
imp_token_len = 0
imp_tokens = [nltk.word_tokenize(t) for t in imperative_sentence]

for i in imp_tokens:
    imp_token_len += len(i)

imp_avg_token = imp_token_len / len(imp_tokens)

print(imp_avg_token)

20.35226264418811


In [10]:
########################################################################################################

In [50]:
# Create table to show data
from prettytable import PrettyTable
table = PrettyTable()
table.field_names = ["Type", "Numbers", "Avg Length (Characters)", "Avg Length (Tokens)", "Percentage in Total"]
table.add_row(["Question Sentence", len(question_sentence), ques_avg_len, ques_avg_token, str(len(question_sentence)/len(textsplit)*100) + "%"])
table.add_row(["Imperative Sentence", len(imperative_sentence), imp_avg_len, imp_avg_token, str(len(imperative_sentence)/len(textsplit)*100) + "%"])

print(table)

+---------------------+---------+-------------------------+---------------------+----------------------+
|         Type        | Numbers | Avg Length (Characters) | Avg Length (Tokens) | Percentage in Total  |
+---------------------+---------+-------------------------+---------------------+----------------------+
|  Question Sentence  |   3682  |    136.81966322650734   |  30.422053231939163 | 0.4056960519008877%  |
| Imperative Sentence |   1127  |    90.41437444543034    |  20.35226264418811  | 0.12417692843354165% |
+---------------------+---------+-------------------------+---------------------+----------------------+


In [12]:
# Word Tokenizer

# ==== Question ====    
question_tokens = []
for t in question_sentence:
    question_tokens += nltk.word_tokenize(t) 
    
# ==== Imperative ====    
imperative_tokens = []
for t in imperative_sentence:
    imperative_tokens += nltk.word_tokenize(t) 
    
print("-----Question:")
print(len(question_tokens))
print(question_tokens[:20])
print()
print("-----Imperative:")
print(len(imperative_tokens))
print(imperative_tokens[:20])

-----Question:
112014
['I', 'would', 'definitely', 'try', 'other', 'languages', 'after', 'seeing', 'just', 'how', 'easy', 'it', 'is', 'to', 'use', 'this', 'stuff.What', 'did', "n't", 'I']

-----Imperative:
22937
['Fits', 'great', 'and', 'easy', 'to', 'clean', '!', 'Ca', "n't", 'complain', 'about', 'these', 'shoes', 'at', 'all', 'except', 'that', 'I', "'m", 'worried']


In [13]:
# Lowercase 

# ==== Question ====
question_lowercase = [w.lower() for w in question_tokens] 

# ==== Imperative ====
imperative_lowercase = [w.lower() for w in imperative_tokens] 

print("-----Question:")
print(len(question_lowercase))
print(question_lowercase[:20])
print()
print("-----Imperative:")
print(len(imperative_lowercase))
print(imperative_lowercase[:20])

-----Question:
112014
['i', 'would', 'definitely', 'try', 'other', 'languages', 'after', 'seeing', 'just', 'how', 'easy', 'it', 'is', 'to', 'use', 'this', 'stuff.what', 'did', "n't", 'i']

-----Imperative:
22937
['fits', 'great', 'and', 'easy', 'to', 'clean', '!', 'ca', "n't", 'complain', 'about', 'these', 'shoes', 'at', 'all', 'except', 'that', 'i', "'m", 'worried']


In [14]:
# alphabets 

# ==== Question ====
question_alphaWords = [w for w in question_lowercase if w.isalpha()]

# ==== Imperative ====
imperative_alphaWords = [w for w in imperative_lowercase if w.isalpha()]


print("-----Question:")
print(len(question_alphaWords))
print(question_alphaWords[:20])

print()
print("-----Imperative:")
print(len(imperative_alphaWords))
print(imperative_alphaWords[:20])

-----Question:
91179
['i', 'would', 'definitely', 'try', 'other', 'languages', 'after', 'seeing', 'just', 'how', 'easy', 'it', 'is', 'to', 'use', 'this', 'did', 'i', 'like', 'for']

-----Imperative:
18609
['fits', 'great', 'and', 'easy', 'to', 'clean', 'ca', 'complain', 'about', 'these', 'shoes', 'at', 'all', 'except', 'that', 'i', 'worried', 'about', 'getting', 'them']


In [15]:
# Lemmatization
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

start = time.time()

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

# Init Lemmatizer
lemmatizer = WordNetLemmatizer()

# ==== Question ====
question_lemmatizedWords = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in question_alphaWords]

# ==== Imperative ====
imperative_lemmatizedWords = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in imperative_alphaWords]


print('time:\t ' , time.time()-start)

print("-----Question:")
print(len(question_lemmatizedWords))
print(question_lemmatizedWords[:20])

print()
print("-----Imperative:")
print(len(imperative_lemmatizedWords))
print(imperative_lemmatizedWords[:20])

time:	  20.58369517326355
-----Question:
91179
['i', 'would', 'definitely', 'try', 'other', 'language', 'after', 'see', 'just', 'how', 'easy', 'it', 'be', 'to', 'use', 'this', 'do', 'i', 'like', 'for']

-----Imperative:
18609
['fit', 'great', 'and', 'easy', 'to', 'clean', 'ca', 'complain', 'about', 'these', 'shoe', 'at', 'all', 'except', 'that', 'i', 'worried', 'about', 'get', 'them']


In [16]:
# Stop word list 
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 

# ==== Question ====
question_filtered_words = [w for w in question_lemmatizedWords if not w in stop_words ]

# ==== Imperative ====
imperative_filtered_words = [w for w in imperative_lemmatizedWords if not w in stop_words ]

print("-----Question:")
print(len(question_filtered_words))
print(question_filtered_words[:20])

print()
print("-----Imperative:")
print(len(imperative_filtered_words))
print(imperative_filtered_words[:20])

-----Question:
42504
['would', 'definitely', 'try', 'language', 'see', 'easy', 'use', 'like', 'starter', 'headphone', 'microphone', 'come', 'quality', 'comfortable', 'ear', 'get', 'really', 'hot', 'slightly', 'disappointed']

-----Imperative:
9565
['fit', 'great', 'easy', 'clean', 'ca', 'complain', 'shoe', 'except', 'worried', 'get', 'dirty', 'love', 'itwas', 'pr', 'auml', 'sentit', 'verry', 'niceevery', 'bodyliked', 'itand']


In [17]:
# frequency distribution with FreqDist
from nltk import FreqDist

# ==== Question ====
question_fdist = FreqDist(question_filtered_words)
question_fdistkeys = list(question_fdist.keys())

# ==== Imperative ====
imperative_fdist = FreqDist(imperative_filtered_words)
imperative_fdistkeys = list(imperative_fdist.keys())

print("-----Question:")
print(len(question_fdistkeys))
print(question_fdistkeys[:20])
print()
print("-----Imperative:")
print(len(imperative_fdistkeys))
print(imperative_fdistkeys[:20])

-----Question:
5069
['would', 'definitely', 'try', 'language', 'see', 'easy', 'use', 'like', 'starter', 'headphone', 'microphone', 'come', 'quality', 'comfortable', 'ear', 'get', 'really', 'hot', 'slightly', 'disappointed']

-----Imperative:
1891
['fit', 'great', 'easy', 'clean', 'ca', 'complain', 'shoe', 'except', 'worried', 'get', 'dirty', 'love', 'itwas', 'pr', 'auml', 'sentit', 'verry', 'niceevery', 'bodyliked', 'itand']


In [18]:
########### Unigram ###########

In [19]:
# 30 words by frequency

# ==== Question ====
print("-----Question:")
question_topkeys = question_fdist.most_common(30)
for pair in question_topkeys:
    print (pair)
    
# ==== Imperative ====
print()
print("-----Imperative:")
imperative_topkeys = imperative_fdist.most_common(30)
for pair in imperative_topkeys:
    print (pair)

-----Question:
('look', 528)
('like', 522)
('size', 481)
('would', 463)
('get', 443)
('wear', 415)
('fit', 414)
('say', 401)
('make', 377)
('shoe', 376)
('one', 348)
('good', 343)
('well', 334)
('great', 323)
('could', 300)
('love', 273)
('really', 258)
('comfortable', 248)
('want', 248)
('watch', 242)
('go', 237)
('color', 235)
('know', 227)
('order', 226)
('small', 224)
('much', 219)
('little', 218)
('price', 215)
('pair', 209)
('right', 205)

-----Imperative:
('look', 329)
('great', 260)
('fit', 189)
('get', 139)
('love', 118)
('size', 118)
('comfortable', 116)
('like', 115)
('wear', 112)
('make', 109)
('good', 107)
('go', 99)
('nice', 94)
('would', 88)
('well', 85)
('shoe', 79)
('order', 77)
('cute', 75)
('buy', 62)
('perfect', 62)
('ca', 59)
('price', 59)
('sure', 58)
('really', 57)
('recommend', 55)
('color', 53)
('bought', 53)
('small', 53)
('shirt', 51)
('pair', 50)


In [20]:
import re 

def alpha_filter(w):
    pattern = re.compile('^[a-z]+$')
    if (pattern.match(w)):
        return False
    else:
        return True

In [21]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()

# ==== Question ====
print("-----Question:")
question_finder = BigramCollocationFinder.from_words(question_lowercase)
question_scored = question_finder.score_ngrams(bigram_measures.raw_freq)

for bscore in question_scored[:20]:
    print(bscore)
    
# ==== Imperative ====
print()
print("-----Imperative:")
imperative_finder = BigramCollocationFinder.from_words(imperative_lowercase)
imperative_scored = imperative_finder.score_ngrams(bigram_measures.raw_freq)

for bscore in imperative_scored[:20]:
    print(bscore)

-----Question:
((',', 'i'), 0.00562429696287964)
((',', 'but'), 0.0044994375703037125)
(('?', 'i'), 0.004204831538914778)
(('?', '?'), 0.0034638527326941275)
(('?', ','), 0.003392433088721053)
((',', 'and'), 0.0031781741568018282)
(('&', '#'), 0.0031067545128287536)
(('#', '34'), 0.0029014230364061634)
(('34', ';'), 0.0029014230364061634)
(('it', "'s"), 0.0025443248165407896)
(('in', 'the'), 0.002508614994554252)
(('they', 'are'), 0.0023925580730980054)
(('of', 'the'), 0.0022586462406484903)
(('?', ')'), 0.0022140089631653187)
(('what', 'more'), 0.0021872265966754157)
(('do', "n't"), 0.001964040209259557)
(('i', "'m"), 0.0019194029317763852)
(('it', 'is'), 0.0018390558323066759)
(('?', 'the'), 0.0018301283768100416)
(('?', 'what'), 0.0018122734658167728)

-----Imperative:
(('!', ','), 0.01813663513101103)
(('!', '!'), 0.007760387147403758)
((',', 'i'), 0.007673191786196974)
(('!', 'looks'), 0.004577756463356149)
((',', 'and'), 0.0037929982124950954)
(('great', '!'), 0.00370580285128831

In [22]:
# alpha bigrams

# ==== Question ====
print("-----Question:")
question_finder.apply_word_filter(alpha_filter)
question_scored1 = question_finder.score_ngrams(bigram_measures.raw_freq)
for bscore in question_scored1[:20]:
    print(bscore)
    
# ==== Imperative ====
print()
print("-----Imperative:")
imperative_finder.apply_word_filter(alpha_filter)
imperative_scored1 = imperative_finder.score_ngrams(bigram_measures.raw_freq)
for bscore in imperative_scored1[:20]:
    print(bscore)

-----Question:
(('in', 'the'), 0.002508614994554252)
(('they', 'are'), 0.0023925580730980054)
(('of', 'the'), 0.0022586462406484903)
(('what', 'more'), 0.0021872265966754157)
(('it', 'is'), 0.0018390558323066759)
(('i', 'have'), 0.0015980145338975486)
(('to', 'be'), 0.0015890870784009141)
(('a', 'little'), 0.0014998125234345708)
(('and', 'i'), 0.0014730301569446677)
(('for', 'a'), 0.001455175245951399)
(('is', 'a'), 0.0013748281464816899)
(('for', 'the'), 0.0013569732354884211)
(('and', 'the'), 0.0013480457799917868)
(('i', 'am'), 0.0013480457799917868)
(('on', 'the'), 0.0013480457799917868)
(('can', 'you'), 0.0013123359580052493)
(('i', 'say'), 0.0013123359580052493)
(('can', 'i'), 0.0012676986805220775)
(('if', 'you'), 0.0012319888585355402)
(('i', 'was'), 0.0011427143035691967)

-----Imperative:
(('looks', 'great'), 0.0034006190870645684)
(('they', 'are'), 0.002572263155600122)
(('in', 'the'), 0.0023978724331865544)
(('for', 'my'), 0.002179884030169595)
(('i', 'love'), 0.00187470026

In [23]:
########### Bigram ###########

In [24]:
# stopword bigrams  -> Bigrams Result

# ==== Question ====
print("-----Question:")
question_finder.apply_word_filter(lambda w: w in stop_words)
question_scored2 = question_finder.score_ngrams(bigram_measures.raw_freq)
for bscore in question_scored2[:20]:
    print(bscore)
    
# ==== Imperative ====
print()
print("-----Imperative:")
imperative_finder.apply_word_filter(lambda w: w in stop_words)
imperative_scored2 = imperative_finder.score_ngrams(bigram_measures.raw_freq)
for bscore in imperative_scored2[:20]:
    print(bscore)

-----Question:
(('go', 'wrong'), 0.00031246094238220225)
(('well', 'made'), 0.00028567857589229916)
(('year', 'old'), 0.00024104129840912743)
(('look', 'like'), 0.00023211384291249308)
(('good', 'quality'), 0.0001964040209259557)
(('looks', 'great'), 0.0001964040209259557)
(('would', 'recommend'), 0.00018747656542932134)
(('feel', 'like'), 0.000178549109932687)
(('another', 'pair'), 0.00016962165443605264)
(('high', 'quality'), 0.0001606941989394183)
(('make', 'sure'), 0.0001606941989394183)
(('highly', 'recommend'), 0.00015176674344278393)
(('fits', 'great'), 0.00014283928794614958)
(('little', 'bit'), 0.00014283928794614958)
(('looks', 'like'), 0.00014283928794614958)
(('much', 'better'), 0.00014283928794614958)
(('size', 'smaller'), 0.00013391183244951523)
(('years', 'ago'), 0.00013391183244951523)
(('look', 'good'), 0.00012498437695288088)
(('one', 'size'), 0.00012498437695288088)

-----Imperative:
(('looks', 'great'), 0.0034006190870645684)
(('fits', 'great'), 0.001307930418101757